In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# from google.colab import auth
# auth.authenticate_user()

In [ ]:
# from googleapiclient.discovery import build

# # Build the Drive service
# service = build('drive', 'v3')

In [ ]:
import pandas as pd
import numpy as np
import os
from collections import Counter


In [ ]:
root = "/content/drive/MyDrive/Colposcopy/Data/Patient records"

In [ ]:
path = "/content/drive/MyDrive/Colposcopy/Data/Patient records/Abbas/10.7060/10.706.xlsx"
path = "/content/drive/MyDrive/Colposcopy/Data/Patient records/Yasin/17.656/finaleeee.xlsx"
path = "/content/drive/MyDrive/Colposcopy/Data/Patient records/Cropped Folder/15.849/15.849.xlsx"
f = pd.read_excel(path, keep_default_na=False,
    na_values=[""], dtype= str)

In [ ]:
f

,Colposcopy Standards Procedure Note,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,...,Unnamed: 13,Unnamed: 14,Unnamed: 15,Unnamed: 16,Unnamed: 17,Unnamed: 18,Unnamed: 19,Unnamed: 20,Unnamed: 21,Unnamed: 22
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Date:,Name:,MRN:,Indication for colposcopy:,Contraception:,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1401.12,مرضیه دهقان زاد,NaN,HPV+,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Age:,DOB:,Referred by:,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,38,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,Phone:,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,*09120047013,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,History,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:

f.iloc[9,0]

'History'

In [ ]:
extra_features_columns = ["Age", "Smoking", "Drink", "SD", "Married", "#Partner", "Pop"]
def extract_extra_features(ex_file):
    extra_df = pd.DataFrame(data= np.array([[np.nan]*len(extra_features_columns)]), columns=extra_features_columns)
    age_idx = 3
    if "Age" not in ex_file.iloc[age_idx,0]:
        i=0
        while "Age" not in ex_file.iloc[i,0]:
          i += 1
        age_idx = i
    smoke_idx = 22
    if "Smoking" not in ex_file.iloc[smoke_idx,0]:
        i=0
        while "Smoking" not in ex_file.iloc[i,0]:
          i += 1
        smoke_idx = i

    extra_df['Age'] = ex_file.iloc[age_idx+1,0]
    extra_df['Smoking'] = ex_file.iloc[smoke_idx+1,0]
    extra_df['Drink'] = ex_file.iloc[smoke_idx+1,1]
    extra_df['SD'] = ex_file.iloc[smoke_idx+1,2]
    extra_df['Married'] = ex_file.iloc[smoke_idx+1,3]
    extra_df['#Partner'] = ex_file.iloc[smoke_idx+1,4]

    return extra_df

In [ ]:
d = extract_extra_features(f)
d

,Age,Smoking,Drink,SD,Married,#Partner,Pop
0,38,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
procedure_columns = ["Date", "Adeno:conclusive", "AA:Color", "AA:Margin of aceto acid", "AA:Surface", "AA:Size", "Vessels:Punctuation",
                     "Vessels:Mosaics", "Vessels:cufed gland", "Vessels:Atypical", "Vessels:tree like vessels(on nabotian cyst)", "Logul",
                     "Erossion:lesion", "Erossion:clock", "Location of BX:BX1",
                     "Location of BX:BX2", "Diagnosis:Impression", "Diagnosis:BX1", "Diagnosis:BX2", "Diagnosis:ECC", "Diagnosis:POLYPS",
                     "Diagnosis:INFLMATION", "Diagnosis:ATROPHIC", "EXTRA INFO"]
def extract_procedure(ex_file):
    if ex_file.iloc[26,0] == "Procedure":
      pro_idx = 26
    else:
        i = 0
        while ex_file.iloc[i,0] != "Procedure":
          i += 1
        pro_idx = i
    i = 1
    while not pd.isna(ex_file.iloc[pro_idx+3+i,0]):
      i += 1
    # print(ex_file.iloc[pro_idx+3:pro_idx+3+i, :24].shape[1])
    data = ex_file.iloc[pro_idx+3:pro_idx+3+i, :24].values
    # print(data.shape, len(data))
    if(data.shape[-1]==23):
      data = np.concat((data, np.array([[np.nan]]*data.shape[0])), axis=1)
    # print(data.shape, data)
    procedure_df = pd.DataFrame(data= data, columns=procedure_columns)
    return procedure_df

In [ ]:
s = extract_procedure(f)
s

,Date,Adeno:conclusive,AA:Color,AA:Margin of aceto acid,AA:Surface,AA:Size,Vessels:Punctuation,Vessels:Mosaics,Vessels:cufed gland,Vessels:Atypical,...,Location of BX:BX1,Location of BX:BX2,Diagnosis:Impression,Diagnosis:BX1,Diagnosis:BX2,Diagnosis:ECC,Diagnosis:POLYPS,Diagnosis:INFLMATION,Diagnosis:ATROPHIC,EXTRA INFO
0,1404.02,NO,PINK,NO,None,None,None,None,None,None,...,NaN,NaN,NORMAL,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
s['Diagnosis:Impression'].isna().item()

False

In [ ]:
def fill_normals(pro_data):
    flag = False
    if not pro_data['Diagnosis:Impression'].isna().item() and pro_data["Diagnosis:BX1"].isna().item():
        if pro_data['Diagnosis:Impression'][0].lower() == "normal": flag = True
    if not pro_data["Diagnosis:BX1"].isna().item():
        if pro_data["Diagnosis:BX2"].isna().item():
            if pro_data["Diagnosis:BX1"][0].lower() == "normal": flag = True
        else:
            if pro_data["Diagnosis:BX2"][0].lower() == "normal" and pro_data["Diagnosis:BX2"].item().lower() == "normal" : flag = True
    # print(flag)

    if flag:
        if pro_data['Adeno:conclusive'].isna().item():
            pro_data['Adeno:conclusive'] = "No"
        if pro_data['AA:Color'].isna().item():
            pro_data['AA:Color'] = "Pink"
        for cal in ["AA:Margin of aceto acid", "AA:Surface", "AA:Size", "Vessels:Punctuation", "Vessels:Mosaics", "Erossion:lesion"]:
            if pro_data[cal].isna().item():
                pro_data[cal] = "None"
        for cal in ["Vessels:cufed gland", "Vessels:Atypical", "Vessels:tree like vessels(on nabotian cyst)"]:
            if pro_data[cal].isna().item():
                pro_data[cal] = "No"
        if pro_data['Logul'].isna().item():
            pro_data['Logul'] = "Negative"
    return pro_data

In [ ]:
d = fill_normals(s)
d

,Date,Adeno:conclusive,AA:Color,AA:Margin of aceto acid,AA:Surface,AA:Size,Vessels:Punctuation,Vessels:Mosaics,Vessels:cufed gland,Vessels:Atypical,...,Location of BX:BX1,Location of BX:BX2,Diagnosis:Impression,Diagnosis:BX1,Diagnosis:BX2,Diagnosis:ECC,Diagnosis:POLYPS,Diagnosis:INFLMATION,Diagnosis:ATROPHIC,EXTRA INFO
0,1404.02,NO,PINK,NO,None,None,None,None,None,None,...,NaN,NaN,NORMAL,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
# def extract_HPV_types(ex_file):
#     if ex_file.iloc[9,0] == "History":
#         hist_idx = 9
#     else:
#         i = 0
#         while ex_file.iloc[i,0] != "History":
#           i += 1
#         hist_idx = i
#     i = hist_idx+2
#     hpv_col = []

#     # print(ex_file.iloc[i])
#     while not pd.isna(ex_file.iloc[i,0]):
#         # print((ex_file.iloc[i,0] == np.nan), type(ex_file.iloc[i,0]), "ffffffffff")
#         hpv = ex_file.iloc[i,2]
#         # print(hpv, type(hpv), hpv != np.nan)
#         if not pd.isna(hpv):
#             hpv = hpv.replace(" ", "")
#             if ',' in hpv:
#                 hpv = hpv.split(",")
#             elif '.' in hpv:
#                 hpv = hpv.split(".")
#             elif hpv.isdigit():
#                 hpv = [hpv]
#             else: hpv = [hpv]
#         else:
#             hpv = []
#         hpv_col += hpv
#         i += 1
#     return hpv_col

In [ ]:
# hpv_col = []
# hpv_dict = {}
# for folder in os.listdir(os.path.join(root, "Cropped Folder")):
#     path = os.path.join(root, "Cropped Folder", folder)
#     if not os.path.isdir(path):
#         continue
#     records = os.listdir(path)
#     ex_file = None
#     for record in records:
#         if record.endswith(".xlsx"):
#             ex_name = record
#             try:
#                 ex_file = pd.read_excel(os.path.join(path, record), keep_default_na=False, na_values=[""], dtype= str)
#             except:
#                 ex_file = None
#                 continue
#             # print(path)

#             hpv = extract_HPV_types(ex_file)
#             hpv_col += hpv
#     hpv_dict[folder] = hpv

In [ ]:
# for folder in hpv_dict:
#     if '18.45.56' in hpv_dict[folder]:
#         print(folder, '18.45.56')
#     if 'OTHERS16' in hpv_dict[folder]:
#         print(folder, 'OTHERS16')
#     if '35و53' in hpv_dict[folder]:
#         print(folder, '35و53')

In [ ]:
# hpv_col = ['3','6','11','14','16','18','26','31','32','33','35','36','39','40','41','42','43','44','45','51','52','53','54','55','56','58','59','61','62','66','67','68','70','72','73','74','75','80','81','82','83','84','89','90','91', 'others']

# HPV = pd.DataFrame(data= np.array([[0]*len(hpv_col)]), columns=hpv_col)

# HPV.loc[0,'3']

In [ ]:
hpv_col = ['3','6','11','14','16','18','26','31','32','33','35','36','39','40','41','42','43','44','45','51','52','53','54','55','56','58','59','61','62','66','67','68','70','72','73','74','75','80','81','82','83','84','89','90','91', 'others']
def extract_HPV_Pop(ex_file, date= None):
    HPV = pd.DataFrame(data= np.array([[0]*len(hpv_col)]), columns=hpv_col, dtype= float)
    if ex_file.iloc[9,0] == "History":
        hist_idx = 9
    else:
        i = 0
        while ex_file.iloc[i,0] != "History":
          i += 1
        hist_idx = i
    i = hist_idx+2
    # print(date, "gggggggggggggg")
    date_idx = -1
    while not pd.isna(ex_file.iloc[i,0]):
        if ex_file.iloc[i,0] == date:
            date_idx = i
            break
        i += 1

    if date_idx == -1:
        hpv = [np.nan]
        pop = np.nan
    else:
      hpv = ex_file.iloc[date_idx,2]
      pop = ex_file.iloc[date_idx,1]

    if not pd.isna(hpv):
        hpv = hpv.replace(" ", "")
        if ',' in hpv:
            hpv = hpv.split(",")
        elif '.' in hpv:
            hpv = hpv.split(".")
        elif hpv.isdigit():
            hpv = [hpv]
        elif 'oth' in hpv.lower() or 'pos' in hpv.lower():
            hpv = ['others']
        elif 'neg' in hpv.lower() or 'mal' in hpv.lower():
            hpv = []
        else:
            print(folder, hpv)
    else:
        hpv = [np.nan]

    for h in hpv:
        if pd.isna(h):
            HPV.loc[:,:] = np.nan
        elif h.lower() == 'others':
            HPV.loc[0, 'others'] += 1
        elif h.isdigit() and not h in hpv_col:
            print(h , ' is not in the list')
            HPV.loc[0, 'others'] += 1
        else:
            HPV.loc[0,h] = 1
    return pop, HPV

In [ ]:
# for folder in os.listdir(os.path.join(root, "Cropped Folder")):
#     path = os.path.join(root, "Cropped Folder", folder)
#     if not os.path.isdir(path):
#         continue
#     records = os.listdir(path)
#     ex_file = None
#     for record in records:
#         if record.endswith(".xlsx"):
#             ex_name = record
#             try:
#                 ex_file = pd.read_excel(os.path.join(path, record), keep_default_na=False, na_values=[""], dtype= str)
#             except:
#                 ex_file = None
#                 continue
#             pro_data = extract_procedure(ex_file)
#             # pro_data = fill_normals(pro_data)
#             pop, hpv = extract_HPV_Pop(ex_file, pro_data['Date'][0])

#             # hpv = extract_HPV_types(ex_file)
#             # hpv_col += hpv
#     # hpv_dict[folder] = hpv

In [ ]:
# data_list = []
# for owner in ["Abbas", "Yasin", "Ali", "Bafrin", "Deniz", "Parisa.R", "Parisa.mohammadi"]:
#     data_list += os.listdir(os.path.join(root, owner))
# labeled_data_list = os.listdir(os.path.join(root, "Labeled Folder"))
croppeded_data_list = os.listdir(os.path.join(root, "Cropped Folder"))

In [ ]:
import re
duplicate_pattern = re.compile(r".*\(\d+\)$")

true_croppeded_data_list = []#d for d in croppeded_data_list if not duplicate_pattern.match(d)]
duplicated_croppeded_data_list = []#d for d in croppeded_data_list if duplicate_pattern.match(d)]
for d in croppeded_data_list:
    if '(' in d:
        duplicated_croppeded_data_list.append(d)
    else:
        true_croppeded_data_list.append(d)

In [ ]:
# duplcated_data_list = []

# counts = Counter(data_list)
# duplcated_data_list = {k:v for k, v in counts.items() if v > 1}

# counts = Counter(labeled_data_list)
# duplcated_labeled_data_list = {k:v for k, v in counts.items() if v > 1}

# counts = Counter(croppeded_data_list)
# duplcated_croppeded_data_list = {k:v for k, v in counts.items() if v > 1}

In [ ]:
# print(duplcated_data_list)
# print(duplcated_labeled_data_list)
print(duplicated_croppeded_data_list)

['19.700 (1)', '19.715 (1)', '18.855 (1)', '19.635 (1)', '19.652 (1)', '17 (1).738', '19 (1).681', '18 (1).901', '16 (1).274', '19 (1).50', '18 (1).210', '19 (2).50', '19 (1).35', '18 (1).216', '19.364 (1)', '19 (1).364']


In [ ]:
# not_labeled_data_list = list(set(data_list) - set(labeled_data_list))
# renamed_after_labeling = list(set(labeled_data_list) - set(data_list))

In [ ]:

# print(f"{len(not_labeled_data_list)} of {len(data_list)} are not labeld")
# print(f"{len(renamed_after_labeling)} of {len(labeled_data_list)} are renamed after labeling")

In [ ]:
# print(len(set(data_list).intersection(set(labeled_data_list))))
# print(len(set(data_list).union(set(labeled_data_list))))

In [ ]:
# not_croppeded_data_list = list(set(labeled_data_list) - set(croppeded_data_list))
# renamed_after_cropping = list(set(croppeded_data_list) - set(labeled_data_list))

In [ ]:
# print(f"{len(not_croppeded_data_list)} of {len(labeled_data_list)} are not croppeded")
# print(f"{len(renamed_after_cropping)} of {len(croppeded_data_list)} are renamed after cropping")

In [ ]:
# print(len(set(labeled_data_list).intersection(set(croppeded_data_list))))
# print(len(set(labeled_data_list).union(set(croppeded_data_list))))

In [ ]:
# not_croppeded_data_list += ["None"] * (len(renamed_after_cropping) - len(not_croppeded_data_list))

In [ ]:
# print(np.array([not_croppeded_data_list,renamed_after_cropping]).T)

In [ ]:
# from genericpath import isdir
# OK_data = list(set(labeled_data_list).intersection(set(croppeded_data_list)))
# not_good = []
# prepared_data=[]
# data_columns = ["Patient ID", "jpg_file", "xlsx_file", "Impression", "BX1", "BX2", "Loc_BX1", "Loc_BX2"]
# bad_data_columns = ["Patient ID", "jpg_file", "xlsx_file", "Impression", "EXTRA INFO"]
# for folder in OK_data:
#     path = os.path.join(root, "Cropped Folder", folder)
#     records = os.listdir(path)
#     jpg_file = None
#     ex_file = None
#     # print(folder)
#     rec_num = 0
#     procedure_flag = False
#     for record in records:
#         jpg_file_null = None
#         if os.path.isdir(os.path.join(path, record)) and rec_num < 1:
#             for img in os.listdir(os.path.join(path, record)):
#                 if 'A1' in img:
#                     jpg_file_null = os.path.join(record,img)
#                 elif 'A' in img:
#                     jpg_file_null = os.path.join(record,img)
#         if jpg_file_null is not None:
#             rec_num += 1
#             jpg_file = jpg_file_null

#         if record.endswith(".xlsx"):
#           ex_name = record
#           try:
#               ex_file = pd.read_excel(os.path.join(path, record))
#           except:
#               ex_file = None
#               continue
#           procedure_data = extract_procedure(ex_file)
#           # print(procedure_data['Diagnosis:Impression'].isna(), "555")
#           # print(procedure_data['EXTRA INFO'], "666")
#           procedure_idx = 0
#           for idx in range(procedure_data.shape[0]):
#               if procedure_data['Diagnosis:Impression'].isna()[idx] and procedure_data['Diagnosis:BX1'].isna()[idx] and procedure_data['Diagnosis:BX2'].isna()[idx]:
#                   continue
#               if procedure_data['EXTRA INFO'][idx] == "DELET":
#                   continue
#               procedure_idx = idx
#               procedure_flag = True
#               break
#     if jpg_file is None or ex_file is None:
#         not_good.append([folder, jpg_file, ex_file is None, procedure_data['Diagnosis:Impression'][0], procedure_data['EXTRA INFO'][0]])
#     elif not procedure_flag:
#         not_good.append([folder, jpg_file, ex_file is None, procedure_data['Diagnosis:Impression'][0], procedure_data['EXTRA INFO'][0]])
#     else:
#         prepared_data.append([folder, jpg_file, ex_name, procedure_data['Diagnosis:Impression'][procedure_idx], procedure_data['Diagnosis:BX1'][procedure_idx], procedure_data['Diagnosis:BX2'][procedure_idx],
#                               procedure_data['Location of BX:BX1'][procedure_idx],  procedure_data['Location of BX:BX2'][procedure_idx]])
# prepared_data = pd.DataFrame(data=prepared_data, columns=data_columns)
# not_good = pd.DataFrame(data=not_good, columns=bad_data_columns)

In [ ]:
# from genericpath import isdir
# OK_data = true_croppeded_data_list
# not_good = []
# prepared_data=[]
# data_columns = ["Patient ID", "jpg_file", "xlsx_file", "Impression", "BX1", "BX2", "Loc_BX1", "Loc_BX2"]
# bad_data_columns = ["Patient ID", "jpg_file", "xlsx_file", "Impression", "EXTRA INFO"]
# not_directory = []
# for folder in OK_data:
#     path = os.path.join(root, "Cropped Folder", folder)
#     if not os.path.isdir(path):
#         not_directory.append(folder)
#         continue
#     records = os.listdir(path)
#     jpg_file = None
#     ex_file = None
#     # print(folder)
#     rec_num = 0
#     procedure_flag = False
#     for record in records:
#         jpg_file_null = None
#         if os.path.isdir(os.path.join(path, record)) and rec_num < 1:
#             for img in os.listdir(os.path.join(path, record)):
#                 if 'A1' in img:
#                     jpg_file_null = os.path.join(record,img)
#                 elif 'A' in img:
#                     jpg_file_null = os.path.join(record,img)
#         if jpg_file_null is not None:
#             rec_num += 1
#             jpg_file = jpg_file_null

#         if record.endswith(".xlsx"):
#           ex_name = record
#           try:
#               ex_file = pd.read_excel(os.path.join(path, record))
#           except:
#               ex_file = None
#               continue
#           procedure_data = extract_procedure(ex_file)
#           # print(procedure_data['Diagnosis:Impression'].isna(), "555")
#           # print(procedure_data['EXTRA INFO'], "666")
#           procedure_idx = 0
#           for idx in range(procedure_data.shape[0]):
#               if procedure_data['Diagnosis:Impression'].isna()[idx] and procedure_data['Diagnosis:BX1'].isna()[idx] and procedure_data['Diagnosis:BX2'].isna()[idx]:
#                   continue
#               if procedure_data['EXTRA INFO'][idx] == "DELET":
#                   continue
#               procedure_idx = idx
#               procedure_flag = True
#               break


#     if jpg_file is None or ex_file is None or "D" in folder:
#         not_good.append([folder, jpg_file, ex_file is None, procedure_data['Diagnosis:Impression'][0], procedure_data['EXTRA INFO'][0]])
#     elif not procedure_flag:
#         not_good.append([folder, jpg_file, ex_file is None, procedure_data['Diagnosis:Impression'][0], procedure_data['EXTRA INFO'][0]])
#     else:
#         prepared_data.append([folder, jpg_file, ex_name, procedure_data['Diagnosis:Impression'][procedure_idx], procedure_data['Diagnosis:BX1'][procedure_idx], procedure_data['Diagnosis:BX2'][procedure_idx],
#                               procedure_data['Location of BX:BX1'][procedure_idx],  procedure_data['Location of BX:BX2'][procedure_idx]])
# prepared_data = pd.DataFrame(data=prepared_data, columns=data_columns)
# not_good = pd.DataFrame(data=not_good, columns=bad_data_columns)

In [ ]:
#exclude 19.681: Ipmression == 'Erossion', 'Errosion == None'

In [100]:
# from genericpath import isdir
# OK_data = true_croppeded_data_list
# not_good = []
# data_list = []
# hpv_list = []
# data_columns = ["Patient ID", "jpg_file", "xlsx_file"] + procedure_columns + extra_features_columns + ['Abnormality(Impression)', 'Abnormality(BX)']
# bad_data_columns = ["Patient ID", "jpg_file", "xlsx_file", "Impression", "EXTRA INFO"]
# not_directory = []
# Imp_unique = {'Normal': 'Normal', 'Metaplasia': 'Metaplasia', 'Low grade ': 'Low grade', 'Erossion': 'Erosion', 'Intermadiate': 'Intermediate',
#        'Intermediate': 'Intermediate', 'High grade ': 'High grade', 'Low grade': 'Low grade', 'High grade': 'High grade',
#        'Eversion':'Eversion', 'Cancer ': 'Cancer', 'INTERMEDIAT': 'Intermediate', 'Errosion': 'Erosion', 'EROSION': 'Erosion',
#        'NORMAL': 'Normal', 'LOW GRADE': 'Low grade'}
# for folder in OK_data:
#     path = os.path.join(root, "Cropped Folder", folder)
#     if not os.path.isdir(path):
#         not_directory.append(folder)
#         continue
#     records = os.listdir(path)
#     jpg_file = None
#     ex_file = None
#     # print(folder)
#     rec_num = 0
#     procedure_flag = False
#     for record in records:
#         jpg_file_null = None
#         if os.path.isdir(os.path.join(path, record)) and rec_num < 1:
#             for img in os.listdir(os.path.join(path, record)):
#                 if 'A1' in img:
#                     jpg_file_null = os.path.join(record,img)
#                 elif 'A' in img:
#                     jpg_file_null = os.path.join(record,img)
#         if jpg_file_null is not None:
#             rec_num += 1
#             jpg_file = jpg_file_null

#         if record.endswith(".xlsx"):
#           ex_name = record
#           try:
#               ex_file = pd.read_excel(os.path.join(path, record), keep_default_na=False, na_values=[""], dtype= str)
#           except:
#               ex_file = None
#               continue
#           procedure_data = extract_procedure(ex_file)
#           # print(procedure_data['Diagnosis:Impression'].isna(), "555")
#           # print(procedure_data['EXTRA INFO'], "666")
#           procedure_idx = 0
#           for idx in range(procedure_data.shape[0]):
#               if procedure_data['Diagnosis:Impression'].isna()[idx] and procedure_data['Diagnosis:BX1'].isna()[idx] and procedure_data['Diagnosis:BX2'].isna()[idx]:
#                   continue
#               if procedure_data['EXTRA INFO'][idx] == "DELET":
#                   continue
#               procedure_idx = idx
#               procedure_flag = True
#               break

#     if jpg_file is None or ex_file is None or "D" in folder:
#         not_good.append([folder, jpg_file, ex_file is None, procedure_data['Diagnosis:Impression'][procedure_idx], procedure_data['EXTRA INFO'][procedure_idx]])
#     elif not procedure_flag:
#         not_good.append([folder, jpg_file, ex_file is None, procedure_data['Diagnosis:Impression'][procedure_idx], procedure_data['EXTRA INFO'][procedure_idx]])
#     else:
#         if not pd.isna(procedure_data['Diagnosis:Impression'][procedure_idx]):
#             procedure_data.loc[procedure_idx, 'Diagnosis:Impression'] = Imp_unique[procedure_data['Diagnosis:Impression'][procedure_idx]]
#         extra_feat = extract_extra_features(ex_file)
#         pop, hpv = extract_HPV_Pop(ex_file, procedure_data['Date'][procedure_idx])
#         extra_feat['Pop'] = pop
#         data = [folder, jpg_file, ex_name] + procedure_data.loc[procedure_idx].tolist() + extra_feat.loc[0].tolist() + [np.nan, np.nan]
#         data_list.append(data)
#         hpv_list.append(hpv.loc[0].tolist())

# prepared_data = pd.DataFrame(data=data_list, columns=data_columns)
# hpv_data = pd.DataFrame(data=hpv_list, columns=hpv_col)
# not_good = pd.DataFrame(data=not_good, columns=bad_data_columns)

In [103]:
# flag_imp_normal = (prepared_data["Diagnosis:Impression"] == "Normal") | (prepared_data['Diagnosis:Impression'] == 'Eversion')
# flag_imp_nan = prepared_data["Diagnosis:Impression"].isna()
# flag_bx_nan = prepared_data["Diagnosis:BX1"].isna() & prepared_data["Diagnosis:BX2"].isna()
# flag_bx_normal = (prepared_data['Diagnosis:BX1'] == 'Normal') & (prepared_data['Diagnosis:BX2'].isna() | (prepared_data['Diagnosis:BX2'] == 'Normal'))

# prepared_data['Abnormality(Impression)'] = (~flag_imp_normal & ~flag_imp_nan) | (flag_imp_nan & ~flag_bx_normal)
# prepared_data['Abnormality(BX)'] = (~flag_bx_nan & ~flag_bx_normal) | (flag_bx_nan & ~flag_imp_normal)

In [104]:
# prepared_data.to_excel("/content/drive/MyDrive/Colposcopy/Data/Patient records/Meta/raw_data.xlsx", index=False)
# hpv_data.to_excel("/content/drive/MyDrive/Colposcopy/Data/Patient records/Meta/HPV_data.xlsx", index=False)
# not_good.to_excel("/content/drive/MyDrive/Colposcopy/Data/Patient records/Meta/bad_data.xlsx", index=False)

In [82]:
prepared_data = pd.read_excel("/content/drive/MyDrive/Colposcopy/Data/Patient records/Meta/raw_data.xlsx", keep_default_na=False, na_values=[""], dtype= str)
hpv_data = pd.read_excel("/content/drive/MyDrive/Colposcopy/Data/Patient records/Meta/HPV_data.xlsx", keep_default_na=False, na_values=[""], dtype= str)
not_good = pd.read_excel("/content/drive/MyDrive/Colposcopy/Data/Patient records/Meta/bad_data.xlsx", keep_default_na=False, na_values=[""], dtype= str)

In [105]:
print(f"{len(not_good)} of {len(OK_data)-1} are not good \n")
print(f"Patients {not_good['Patient ID'][not_good['jpg_file'].isna()].values} are without image or not labeled but cropped!\n")
print(f"Patiens {not_good['Patient ID'][not_good['Impression'].isna()].values} are without impression\n")
print(f"Patiens {not_good['Patient ID'][not_good['EXTRA INFO'] == 'DELET'].values} are deleted (EXTRA INFO)\n")

49 of 464 are not good 

Patients ['17.334 D' '17.301 D' '17.690 D' '18.153 D' '18.154 D' '18.203 D'
 '19.4 X' '14.433 D' '17.172 D' '19.551 D' '19.584 D' '19.600 D'
 '19.646 D' '19.684 D' '19.739' '19.128 D' '19.48 D' '19.49 D' '18.741___'
 '18.693___' '18.794___' '19.732___' '17.817' '19.31__' '18.808'
 '15.624 D' '13.603 D' '15.381 D' '15.415 D' '15.602 D' '16.441'] are without image or not labeled but cropped!

Patiens ['17.334 D' '17.301 D' '17.690 D' '18.153 D' '19.551 D' '19.584 D'
 '19.646 D' '19.173' '18.794___' '18.623' '18.627' '17.932' '18.779'
 '18.780' '18.832' '19.509' '17.253' '18.614' '18.613' '18.594' '19.715'
 '13.603 D' '15.341' '16.119'] are without impression

Patiens ['18.623' '18.627' '18.689' '18.983' '18.590' '13.603 D'] are deleted (EXTRA INFO)



In [106]:
print(len(prepared_data))

415


In [107]:
prepared_data.head()

,Patient ID,jpg_file,xlsx_file,Date,Adeno:conclusive,AA:Color,AA:Margin of aceto acid,AA:Surface,AA:Size,Vessels:Punctuation,...,EXTRA INFO,Age,Smoking,Drink,SD,Married,#Partner,Pop,Abnormality(Impression),Abnormality(BX)
0,17.314,17.314/17_314_A3.TIF,17.314.xlsx,1402.12,Yes,Pink,None,None,None,None,...,NaN,42,NaN,NaN,NaN,NaN,NaN,ASCUS,False,False
1,17.310,17.310/17_310_A3.TIF,17.310.xlsx,1402.1,No,Pink,None,None,None,None,...,NaN,29,No,No,23,Yes,1,NILM,True,True
2,17.421,17.421/17_421_A.JPG,17.421.xlsx,1403.02,Yes,Snowy White,Geographical,Flat,Large,None,...,NaN,35,NaN,NaN,NaN,NaN,NaN,NaN,True,False
3,17.303,17.303/17_303_A3.TIF,17.303.xlsx,1402.12,Yes,Pink,None,None,None,None,...,NaN,22,Yes,Yes,20,NaN,2,NaN,False,False
4,17.464,17.464/17_464_A.JPG,17.464.xlsx,1403.12,No,Pink,None,None,None,None,...,NaN,43,No,No,NaN,NaN,4,NILM,True,False


In [108]:
prepared_data.loc[:5, ["Diagnosis:Impression", "Diagnosis:BX1", "Diagnosis:BX2", "Abnormality(Impression)", "Abnormality(BX)"]]

,Diagnosis:Impression,Diagnosis:BX1,Diagnosis:BX2,Abnormality(Impression),Abnormality(BX)
0,Normal,Normal,NaN,False,False
1,Metaplasia,CIN1,NaN,True,True
2,Low grade,Normal,NaN,True,False
3,Normal,NaN,NaN,False,False
4,Erosion,Normal,NaN,True,False
5,Normal,Normal,NaN,False,False


In [86]:
# prepared_data.dropna(subset=['Diagnosis:Impression'], inplace=True)

In [109]:
prepared_data['Diagnosis:Impression'].unique()

array(['Normal', 'Metaplasia', 'Low grade', 'Erosion', 'Intermediate',
       'High grade', 'Eversion', 'Cancer', nan], dtype=object)

In [110]:
prepared_data['Diagnosis:BX1'].unique()

array(['Normal', 'CIN1', nan, 'CIN3', 'CIN2', 'Cancer', 'AIS'],
      dtype=object)

In [111]:
prepared_data['Patient ID'][prepared_data['Diagnosis:BX1'] == 'AIS']

,Patient ID
242,18.838


In [112]:
prepared_data['Diagnosis:BX2'].unique()

array([nan, 'CIN1', 'Normal', 'CIN2', 'CIN3'], dtype=object)

In [113]:
print(f'{len(prepared_data[prepared_data["Diagnosis:Impression"] == "Erosion"])}')

14


In [114]:
flag = (prepared_data["Diagnosis:Impression"] == "Erosion")
print(prepared_data['Erossion:lesion'][flag])
print(prepared_data['Patient ID'][flag][prepared_data['Erossion:lesion'][flag] == 'None'])

4       Yes
53     None
63      Yes
82      Yes
182     Yes
219     Yes
226     Yes
234     Yes
235     Yes
244     Yes
295     Yes
318     Yes
354     Yes
356     Yes
Name: Erossion:lesion, dtype: object
53    19.681
Name: Patient ID, dtype: object


In [115]:
flag = (prepared_data["Diagnosis:Impression"] == "Eversion")
print(prepared_data['Patient ID'][flag])

80      19.10
214    17.779
221    17.806
228    18.746
232    18.784
237    18.833
240    18.824
Name: Patient ID, dtype: object


In [116]:
print(f'Between {len(prepared_data)} data with one of Ipression and BX available:')
print(f"{len(prepared_data[prepared_data["Diagnosis:Impression"].isna()])} have only BX")
print(f"{len(prepared_data[prepared_data["Diagnosis:BX1"].isna()])} have only Impression")
print(f"{len(prepared_data[~prepared_data["Diagnosis:Impression"].isna() & ~prepared_data["Diagnosis:BX1"].isna()])} have both")

print("")

flag_imp = (prepared_data["Diagnosis:Impression"] == "Normal") | (prepared_data['Diagnosis:Impression'] == 'Eversion')
flag_bx =  (~prepared_data['Diagnosis:BX1'].isna() & (prepared_data['Diagnosis:BX1'] != 'Normal')) | (~prepared_data['Diagnosis:BX2'].isna() & (prepared_data['Diagnosis:BX2'] != 'Normal'))
flag = flag_imp & flag_bx
print(np.sum(flag), ' have Normal impression while BX is Abdormal')
# print(prepared_data['Patient ID'][flag])

flag_imp = ~prepared_data["Diagnosis:Impression"].isna() & (prepared_data["Diagnosis:Impression"] != "Normal") & (prepared_data['Diagnosis:Impression'] != 'Eversion')
flag_bx =  (prepared_data['Diagnosis:BX1'] == 'Normal') & (prepared_data['Diagnosis:BX2'].isna() | (prepared_data['Diagnosis:BX2'] == 'Normal'))
flag = flag_imp & flag_bx
print(np.sum(flag), ' have Abnormal impression while BX is Normal')
# print(prepared_data['Patient ID'][flag])

Between 415 data with one of Ipression and BX available:
1 have only BX
89 have only Impression
325 have both

8  have Normal impression while BX is Abdormal
43  have Abnormal impression while BX is Normal


In [117]:
print("Data without BX:")

for i in range(len(prepared_data[prepared_data["Diagnosis:BX1"].isna()])//8 + 1):
    string = ""
    for j in range(8):
        if (i*8 + j >= len(prepared_data[prepared_data["Diagnosis:BX1"].isna()])):
            break
        s = prepared_data[prepared_data["Diagnosis:BX1"].isna()].iloc[i*8+j, 0]
        s = s + " " * (6 - len(s))
        string += s + "  "
    print(string)
    # print("")

Data without BX:
17.303  17.738  17.752  17.864  17.933  18.132  18.873  16.274  
16.545  16.984  19.756  19.488  19.754  19.762  19.933  19.603  
19.605  19.655  19.647  19.718  19.719  19.727  19.722  19.731  
19.737  19.734  19.743  19.747  19.748  19.166  19.125  19.178  
19.202  19.10   19.31   19.47   19.55   19.56   19.59   19.67   
19.109  18.455  17.854  17.800  17.560  19.268  19.264  19.258  
19.249  19.253  19.257  19.256  19.244  19.232  19.213  19.229  
19.228  19.548  17.373  17.779  17.299  17.285  17.806  18.734  
18.806  18.833  18.824  19.758  17.280  18.539  18.387  19.318  
19.626  18.500  19.816  19.810  19.801  19.652  15.681  15.699  
14.954  14.841  15.151  15.158  15.172  19.58   15.849  15.954  
16.523  


In [118]:
np.sum(~prepared_data['Diagnosis:BX1'].isna())
# np.sum(~prepared_data['Diagnosis:BX1'].isna() | ~prepared_data['Diagnosis:BX2'].isna())

np.int64(326)

In [125]:
flag_CIN1_0 = (prepared_data['Diagnosis:BX1'] == 'CIN1') & (prepared_data['Diagnosis:BX2'].isna() | (prepared_data['Diagnosis:BX2'] == 'CIN1') | ((prepared_data['Diagnosis:BX2'] == 'Normal')))
flag_CIN1 = flag_CIN1_0 | (prepared_data['Diagnosis:BX2'] == 'CIN1') & (prepared_data['Diagnosis:BX1'].isna() | (prepared_data['Diagnosis:BX1'] == 'CIN1') | ((prepared_data['Diagnosis:BX1'] == 'Normal')))
flag_CIN2_0 = (prepared_data['Diagnosis:BX1'] == 'CIN2') & (prepared_data['Diagnosis:BX2'] != 'CIN3') & (prepared_data['Diagnosis:BX2'] != 'Cancer')
flag_CIN2 = flag_CIN2_0 | (prepared_data['Diagnosis:BX2'] == 'CIN2') & (prepared_data['Diagnosis:BX1'] != 'CIN3') & (prepared_data['Diagnosis:BX1'] != 'Cancer') & (prepared_data['Diagnosis:BX1'] != 'AIS')
flag_CIN3_0 = ((prepared_data['Diagnosis:BX1'] == 'CIN3') | (prepared_data['Diagnosis:BX1'] == 'AIS')) & (prepared_data['Diagnosis:BX2'] != 'Cancer')
flag_CIN3 = flag_CIN3_0 | (prepared_data['Diagnosis:BX2'] == 'CIN3') & (prepared_data['Diagnosis:BX2'] != 'Cancer')
flag_Cancer = (prepared_data['Diagnosis:BX1'] == 'Cancer') | (prepared_data['Diagnosis:BX2'] == 'Cancer')
flag_normal = (prepared_data['Diagnosis:BX1'] == 'Normal') & (prepared_data['Diagnosis:BX2'].isna() | (prepared_data['Diagnosis:BX2'] == 'Normal'))
flag_nan = prepared_data['Diagnosis:BX1'].isna() & prepared_data['Diagnosis:BX2'].isna()

print(f'Between {np.sum(~prepared_data['Diagnosis:BX1'].isna())} data with BX:')
print(f'{np.sum(flag_normal)} are Normal')
print(f'{np.sum(flag_CIN1)} have CIN1')
print(f'{np.sum(flag_CIN2)} have CIN2')
print(f'{np.sum(flag_CIN3)} have CIN3')
print(f'{np.sum(flag_Cancer)} have Cancer')

print("")

flag_imp = ~((prepared_data["Diagnosis:Impression"] == "Normal") | (prepared_data['Diagnosis:Impression'] == 'Eversion'))
print(f'Between {np.sum(~prepared_data["Diagnosis:BX1"].isna() & flag_imp)} data without Normal impression (Abormal or nan) which have BX:')
print(f'{np.sum(flag_normal & flag_imp)} are Normal')
print(f'{np.sum(flag_CIN1 & flag_imp)} have CIN1')
print(f'{np.sum(flag_CIN2 & flag_imp)} have CIN2')
print(f'{np.sum(flag_CIN3 & flag_imp)} have CIN3')
print(f'{np.sum(flag_Cancer & flag_imp)} have Cancer')

Between 326 data with BX:
146 are Normal
124 have CIN1
16 have CIN2
33 have CIN3
7 have Cancer

Between 215 data without Normal impression (Abormal or nan) which have BX:
43 are Normal
118 have CIN1
16 have CIN2
31 have CIN3
7 have Cancer


In [98]:
# if pro_data['Adeno:conclusive'].isna().item():
#       pro_data['Adeno:conclusive'] = "No"
# if pro_data['AA:Color'].isna().item():
#     pro_data['AA:Color'] = "Pink"
# for cal in ["AA:Margin of aceto acid", "AA:Surface", "AA:Size", "Vessels:Punctuation", "Vessels:Mosaics", "Erossion:lesion"]:
#     if pro_data[cal].isna().item():
#         pro_data[cal] = "None"
# for cal in ["Vessels:cufed gland", "Vessels:Atypical", "Vessels:tree like vessels(on nabotian cyst)"]:
#     if pro_data[cal].isna().item():
#         pro_data[cal] = "No"
# if pro_data['Logul'].isna().item():
#     pro_data['Logul'] = "Negative"
nan_check_cols = ['Adeno:conclusive', 'AA:Color', "AA:Margin of aceto acid", "AA:Surface", "AA:Size", "Vessels:Punctuation", "Vessels:Mosaics", "Erossion:lesion", "Vessels:cufed gland", "Vessels:Atypical", "Vessels:tree like vessels(on nabotian cyst)", "Logul"]
flag = prepared_data[nan_check_cols].isna().any(axis=1)
flag_imp = (prepared_data["Diagnosis:Impression"] == "Normal") | (prepared_data['Diagnosis:Impression'] == 'Eversion')
print(prepared_data['Patient ID'][flag])

30     19.749
37     16.984
50     19.605
63     19.734
70     19.125
        ...  
405    16.550
406    16.578
408    16.560
411    17.005
413    17.795
Name: Patient ID, Length: 117, dtype: object


In [127]:
from sklearn.model_selection import train_test_split
train_nan, test_nan = train_test_split(prepared_data[flag_nan], test_size=0.25, random_state=42)
train_normal, test_normal = train_test_split(prepared_data[flag_normal], test_size=0.25, random_state=42)
train_CIN1, test_CIN1 = train_test_split(prepared_data[flag_CIN1], test_size=0.25, random_state=42)
train_CIN2, test_CIN2 = train_test_split(prepared_data[flag_CIN2], test_size=0.25, random_state=42)
train_CIN3, test_CIN3 = train_test_split(prepared_data[flag_CIN3], test_size=0.25, random_state=42)
train_cancer, test_cancer = train_test_split(prepared_data[flag_Cancer], test_size=0.25, random_state=42)

print("nan    train/test split:  ", len(train_nan), "/", len(test_nan))
print("normal train/test split:  ", len(train_normal), "/", len(test_normal))
print("CIN1   train/test split:  ", len(train_CIN1), "/", len(test_CIN1))
print("CIN2   train/test split:  ", len(train_CIN2), "/", len(test_CIN2))
print("CIN3   train/test split:  ", len(train_CIN3), "/", len(test_CIN3))
print("cancer train/test split:  ", len(train_cancer), "/", len(test_cancer))

nan    train/test split:   66 / 23
normal train/test split:   109 / 37
CIN1   train/test split:   93 / 31
CIN2   train/test split:   12 / 4
CIN3   train/test split:   24 / 9
cancer train/test split:   5 / 2


In [132]:
hpv_train_nan  = hpv_data.loc[train_nan.index]
hpv_test_nan   = hpv_data.loc[test_nan.index]

hpv_train_normal = hpv_data.loc[train_normal.index]
hpv_test_normal  = hpv_data.loc[test_normal.index]

hpv_train_CIN1 = hpv_data.loc[train_CIN1.index]
hpv_test_CIN1  = hpv_data.loc[test_CIN1.index]

hpv_train_CIN2 = hpv_data.loc[train_CIN2.index]
hpv_test_CIN2  = hpv_data.loc[test_CIN2.index]

hpv_train_CIN3 = hpv_data.loc[train_CIN3.index]
hpv_test_CIN3  = hpv_data.loc[test_CIN3.index]

hpv_train_cancer = hpv_data.loc[train_cancer.index]
hpv_test_cancer  = hpv_data.loc[test_cancer.index]


In [135]:
train_df = pd.concat([train_nan, train_normal, train_CIN1, train_CIN2, train_CIN3, train_cancer])
test_df = pd.concat([test_nan, test_normal, test_CIN1, test_CIN2, test_CIN3, test_cancer])

hpv_train_df = pd.concat([
    hpv_train_nan, hpv_train_normal, hpv_train_CIN1,
    hpv_train_CIN2, hpv_train_CIN3, hpv_train_cancer
])

hpv_test_df = pd.concat([
    hpv_test_nan, hpv_test_normal, hpv_test_CIN1,
    hpv_test_CIN2, hpv_test_CIN3, hpv_test_cancer
])

In [136]:
train_df.head()

,Patient ID,jpg_file,xlsx_file,Date,Adeno:conclusive,AA:Color,AA:Margin of aceto acid,AA:Surface,AA:Size,Vessels:Punctuation,...,EXTRA INFO,Age,Smoking,Drink,SD,Married,#Partner,Pop,Abnormality(Impression),Abnormality(BX)
40,19.488,19.488/19_488_A.JPG,19.488.xlsx,1404.04,No,Snowy White,Geographical,Flat,Large,Fine,...,NaN,47,Yes,No,14,No,4,NILM,True,True
237,18.833,18_833/18_833_A1.JPG,18.833.xlsx,1403.11,Yes,Pink,None,None,None,None,...,NaN,36,No,No,26,NaN,2,NILM,False,False
66,19.748,19.748/19_748_A2.JPG,19.748.xlsx,1404.06,Yes,Snowy White,Geographical,Flat,Large,Fine,...,NaN,26,NaN,NaN,22,NaN,2,NaN,True,True
266,18.539,18_539/18_539_A1.JPG,18.539.xlsx,D:1403.09,No,Pink,None,None,None,Coarse,...,NaN,37,Yes,No,32,NaN,5,NaN,False,False
37,16.984,16.984/16_984_A2.JPG,16.984.xlsx,D 1402.10,No,NaN,NaN,NaN,NaN,NaN,...,NaN,28,No,No,23,Yes,1,NaN,True,True


In [152]:
train_df.loc[:4, ["Diagnosis:Impression", "Diagnosis:BX1", "Diagnosis:BX2", "Abnormality(Impression)", "Abnormality(BX)"]]

,Diagnosis:Impression,Diagnosis:BX1,Diagnosis:BX2,Abnormality(Impression),Abnormality(BX)
373,Low grade,CIN1,NaN,True,True
78,Low grade,NaN,NaN,True,True
205,Low grade,CIN1,NaN,True,True
211,Low grade,CIN1,NaN,True,True
280,Normal,Normal,NaN,False,False
...,...,...,...,...,...
237,Eversion,NaN,NaN,False,False
391,Low grade,CIN1,NaN,True,True
258,Normal,CIN1,NaN,False,True
321,Low grade,CIN1,NaN,True,True


In [139]:
# Generate one shuffle order from train_df
shuffle_idx_train = train_df.sample(frac=1, random_state=42).index

# Apply the SAME order to both prepared_data and hpv_data
train_df       = train_df.loc[shuffle_idx_train]
hpv_train_df   = hpv_train_df.loc[shuffle_idx_train]


shuffle_idx_test = test_df.sample(frac=1, random_state=42).index

test_df       = test_df.loc[shuffle_idx_test]
hpv_test_df   = hpv_test_df.loc[shuffle_idx_test]


In [153]:
print(f"Test data:  {len(test_df)}")
print(f"  Abnormal/Normal Ipression based: {np.sum(test_df['Abnormality(Impression)'])}/{np.sum(~test_df['Abnormality(Impression)'])}")
print(f"  Abnormal/Normal BX based: {np.sum(test_df['Abnormality(BX)'])}/{np.sum(~test_df['Abnormality(BX)'])}")

print(f"Train data: {len(train_df)}")
print(f"  Abnormal/Normal Ipression based: {np.sum(train_df['Abnormality(Impression)'])}/{np.sum(~train_df['Abnormality(Impression)'])}")
print(f"  Abnormal/Normal BX based: {np.sum(train_df['Abnormality(BX)'])}/{np.sum(~train_df['Abnormality(BX)'])}")

Test data:  106
  Abnormal/Normal Ipression based: 71/35
  Abnormal/Normal BX based: 58/48
Train data: 309
  Abnormal/Normal Ipression based: 184/125
  Abnormal/Normal BX based: 162/147


In [140]:
# from sklearn.model_selection import train_test_split
# train_df, test_df = train_test_split(prepared_data, test_size=0.25, random_state=42, )

In [141]:
# train_df['Patient ID'][1]

In [144]:
import os
import json
import pandas as pd


In [143]:
train_data_excel_path = os.path.join("/content/drive/MyDrive/Colposcopy/Data/Patient records/Meta", "train_main_features_75.xlsx")
test_data_excel_path = os.path.join("/content/drive/MyDrive/Colposcopy/Data/Patient records/Meta", "test_main_features_75.xlsx")
train_hpv_path = os.path.join("/content/drive/MyDrive/Colposcopy/Data/Patient records/Meta", "train_HPV_features_75.xlsx")
test_hpv_path = os.path.join("/content/drive/MyDrive/Colposcopy/Data/Patient records/Meta", "test_HPV_features_75.xlsx")

In [ ]:
# train_df.to_excel(train_data_excel_path, index=False)
# test_df.to_excel(test_data_excel_path, index=False)
# hpv_train_df.to_excel(train_hpv_path, index=False)
# hpv_test_df.to_excel(test_hpv_path, index=False)

In [ ]:
train_df = pd.read_excel(train_data_excel_path, keep_default_na=False, na_values=[""], dtype= str)
test_df = pd.read_excel(test_data_excel_path, keep_default_na=False, na_values=[""], dtype= str)

In [ ]:
# train_excel_path = os.path.join("/content/drive/MyDrive/Colposcopy/Data/Patient records/Meta", "train_Impression_based.xlsx")
# test_excel_path = os.path.join("/content/drive/MyDrive/Colposcopy/Data/Patient records/Meta", "test_Impression_based.xlsx")
# train_df.to_excel(train_excel_path, index=False)
# test_df.to_excel(test_excel_path, index=False)

In [ ]:
# import pandas as pd
# train_df = pd.read_excel(train_excel_path, dtype= str)
# test_df = pd.read_excel(test_excel_path, dtype= str)

In [154]:
len(train_df), len(test_df)

(309, 106)

In [ ]:
# import json
# import os

# # assume prepared_data is your DataFrame and jpg_file is something like ".jpg"
# meta = {"train": {'coloposcopy':[]}, "test": {'coloposcopy':[]}}


# def build_section(df):
#     section = []
#     for _, row in df.iterrows():
#         cls_name = "default"  # since we don't have class names
#         sample = {
#             "img_path": os.path.join(row["Patient ID"], row["jpg_file"]),
#             "mask_path": "",
#             "cls_name": "coloposcopy",
#             "specie_name": "",
#             "anomaly": int(row["Impression"] != "Normal")
#         }
#         section.append(sample)
#     return section

# meta["train"]['coloposcopy'] = build_section(train_df)
# meta["test"]['coloposcopy'] = build_section(test_df)




In [145]:
# assume prepared_data is your DataFrame and jpg_file is something like ".jpg"
meta = {"train": {'coloposcopy':[]}, "test": {'coloposcopy':[]}}


def build_section(df):
    section = []
    for _, row in df.iterrows():
        cls_name = "default"  # since we don't have class names
        sample = {
            "img_path": os.path.join(row["Patient ID"], row["jpg_file"]),
            "mask_path": "",
            "cls_name": "coloposcopy",
            "specie_name": "",
            "anomaly": int(row["Abnormality(Impression)"])
        }
        section.append(sample)
    return section

In [ ]:
# meta["train"]['coloposcopy'] = build_section(train_df)
# meta["test"]['coloposcopy'] = build_section(test_df)


In [146]:
c = 0
for i in meta["train"]['coloposcopy']:
    c += i['anomaly']
print(c, len(meta["train"]['coloposcopy'])-c)

184 125


In [147]:
c = 0
for i in meta["test"]['coloposcopy']:
    c += i['anomaly']
print(c, len(meta["test"]['coloposcopy'])-c)

71 35


In [155]:
# # # save
# with open("/content/drive/MyDrive/Colposcopy/Data/Patient records/Meta/meta.json", "w", encoding="utf-8") as f:
#     json.dump(meta, f, indent=2, ensure_ascii=False)

# print("✅ meta.json created successfully.")

✅ meta.json created successfully.


In [ ]:
# new_list = ['15.681',
#  '15.685',
#  '15.699',
#  '15.849',
#  '15.954',
#  '16.028',
#  '16.119',
#  '16.141',
#  '16.263',
#  '16.347',
#  '16.389',
#  '16.395',
#  '16.441',
#  '16.445',
#  '16.523',
#  '16.550',
#  '16.560',
#  '16.570',
#  '16.578',
#  '16.594',
#  '16.681',
#  '16.719',
#  '16.801',
#  '16.807',
#  '16.83',
#  '16.864',
#  '16.928',
#  '16.974',
#  '17.005',
#  '17.209',
#  '17.414',
#  '17.487',
#  '17.500',
#  '17.55',
#  '17.795',
#  '17.945',
#  '17.952',
#  '18.109',
#  '18.465',
#  '18.470',
#  '18.474',
#  '18.478',
#  '18.480',
#  '18.482',
#  '18.483']

In [ ]:
# Croped_list = os.listdir(os.path.join(root, "Cropped Folder"))

In [ ]:
# for f in new_list:
#     if f not in Croped_list:
#         print(f)

In [ ]:
# for f in new_list:
#     if f in Croped_list:
#         print(f)